1. https://huggingface.co/blog/gemma
2. Repo model google/gemma-7b-it is gated. You must be authenticated to access it.
3. Create your 'new token' on your huggingface

pip install torch accelerate transformers langchain chroma sentence-transformers langchain_community pypdf

In [1]:
import os
from dotenv import load_dotenv
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, pipeline
load_dotenv()

True

In [2]:
GEMMA_TOKEN = os.getenv("GEMMA_TOKEN")
model = "google/gemma-7b-it"
device="cuda"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model, token=GEMMA_TOKEN, device=device)

quantization_config = GPTQConfig(
     bits=4,
     group_size=128,
     dataset="c4", # the original datasets used in GPTQ paper [‘wikitext2’,‘c4’,‘c4-new’,‘ptb’,‘ptb-new’]
     desc_act=False,
     tokenizer=tokenizer,
     batch_size=1,
)
quantized=False
if quantized:
    model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="google/gemma-7b-it",
                                                 token=GEMMA_TOKEN,
                                                 quantization_config=quantization_config,
                                                 device_map=device
                                                 )
else:
    model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="google/gemma-7b-it", 
                                                token=GEMMA_TOKEN,
                                                torch_dtype=torch.float16,
                                                device_map=device
                                                )

c:\Users\farza\anaconda3\envs\AR-RT-LLM\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

c:\Users\farza\anaconda3\envs\AR-RT-LLM\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\farza\.cache\huggingface\hub\models--google--gemma-7b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [4]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [5]:
messages = [
    {"role": "user", "content": "tell me a joke"},
]
prompt = pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)
print(outputs[0]["generated_text"][len(prompt):])

Why did the tomato turn red?

Because it saw the salad dressing.
